## Robo Diferencial - Equações de Dinâmica

```TODO```

![](../../../images/ex1_robot.png)

In [2]:
## sympy modulo para trabalhar com equações simbólicas
from sympy import *
import numpy as np

In [3]:
# variaveis
x = Function('x'); y = Function('y'); phi = Function('phi');
t,m,J,Lbd,Fx,Fy,N,L,r,tau_l,tau_r,v,omega = symbols('t m J lambda F_x F_y N L r tau_l tau_r v omega')

### Modelo Cinemático

$\begin{bmatrix}\dot{x} \\ \dot{y} \\ \dot{\phi}\end{bmatrix} = 
\begin{bmatrix}\cos(\phi) & 0 \\ \sin(\phi) & 0 \\ 0 & 1 \end{bmatrix}
\begin{bmatrix}v \\ \omega \end{bmatrix}$ 

ou 

$\mathbf{\dot{q}}= \mathbf{S}(q)\mathbf{v}$

Restrições de Movimento

$\mathbf{A}(q) = [- \sin(\phi) \quad \cos(\phi) \quad 0]$


In [4]:
S = Matrix([[cos(phi(t)), 0],[sin(phi(t)), 0],[0, 1]])
S

Matrix([
[cos(phi(t)), 0],
[sin(phi(t)), 0],
[          0, 1]])

In [5]:
v_matrix = Matrix([[v],[omega]])
v_matrix

Matrix([
[    v],
[omega]])

In [35]:
A = Matrix([[-sin(phi(t)), cos(phi(t)), 0]])

### Começando pela equação de lagrange

$\newcommand{\parcial}[3]{\dfrac{\partial^{#1}#2}{\partial #3^{#1}}}
\newcommand{\df}[1]{\,\mathrm{d}#1}
\frac{d}{\df{t}}\left( \parcial{}{\mathcal{L}}{\dot{q}_k}\right)
-\parcial{}{\mathcal{L}}{q_k}
+\tau_{d_k}
= f_k - \sum\limits^{m}_{j=1}\lambda_j a_{jk} $

### Definindo a função $\mathcal{L}$
$\mathcal{L} = \mathcal{T} - \mathcal{V}$

In [7]:
L_functional = 1/2*m*(diff(x(t),t)**2+ diff(y(t),t)**2) + 1/2*J*diff(phi(t),t)**2

In [8]:
L_functional

0.5*J*Derivative(phi(t), t)**2 + 0.5*m*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)

In [9]:
dq1 = diff( diff(L_functional, diff(x(t),t)) ,t)
dq1

1.0*m*Derivative(x(t), (t, 2))

In [10]:
dq2 = diff( diff(L_functional, diff(y(t),t)) ,t)
dq2

1.0*m*Derivative(y(t), (t, 2))

In [11]:
dq3 = diff( diff(L_functional, diff(phi(t),t)) ,t)
dq3

1.0*J*Derivative(phi(t), (t, 2))

In [12]:
q1 = diff(L_functional, x(t))
q1

0

In [13]:
q2 = diff(L_functional, y(t))
q2

0

In [14]:
q3 = diff(L_functional, phi(t))
q3

0

In [15]:
## N torque
Eq(J*diff(diff(phi(t),t),t),N)

Eq(J*Derivative(phi(t), (t, 2)), N)

In [16]:
# Matriz 
M_matrix = Matrix([[m, 0, 0],[0, m , 0],[0, 0, J]])
M_matrix

Matrix([
[m, 0, 0],
[0, m, 0],
[0, 0, J]])

### Matriz das Forças generalizadas ( $f_k$)

$F = F_r + F_l$

$\tau_r = rF_r$ e $\tau_l = rF_l$

temos

$F = \frac{1}{r}(\tau_l + \tau_r)$

sendo

$F_x = \frac{1}{r}(\tau_l + \tau_r)\cos(\phi)$

e

$F_y = \frac{1}{r}(\tau_l + \tau_r)\sin(\phi)$

In [17]:
E = 1/r*Matrix([[cos(phi(t)),cos(phi(t))], [sin(phi(t)),sin(phi(t))], [L/2, -L/2]])
E

Matrix([
[cos(phi(t))/r, cos(phi(t))/r],
[sin(phi(t))/r, sin(phi(t))/r],
[      L/(2*r),      -L/(2*r)]])

### Restrições

$\mathbf{M}(q)\ddot{\mathbf{q}}=\mathbf{E}(q)\boldsymbol{\tau}-\mathbf{A}(q)\lambda$

In [39]:
Eq(M_matrix*diff(diff(Matrix([[x(t)],[y(t)],[phi(t)]]),t),t),E*Matrix([[tau_r],[ tau_l]])-A.T*Lbd)

Eq(Matrix([
[  m*Derivative(x(t), (t, 2))],
[  m*Derivative(y(t), (t, 2))],
[J*Derivative(phi(t), (t, 2))]]), Matrix([
[ lambda*sin(phi(t)) + tau_l*cos(phi(t))/r + tau_r*cos(phi(t))/r],
[-lambda*cos(phi(t)) + tau_l*sin(phi(t))/r + tau_r*sin(phi(t))/r],
[                                 -L*tau_l/(2*r) + L*tau_r/(2*r)]]))

### Pseudo Velocidades

$\mathbf{\tilde{M}}, \mathbf{\tilde{V}}, \mathbf{\tilde{E}}$

In [32]:
M_tilde = simplify(S.T*M_matrix*S)
M_tilde

Matrix([
[m, 0],
[0, J]])

In [33]:
E_tilde = simplify(S.T*E)
E_tilde

Matrix([
[    1/r,      1/r],
[L/(2*r), -L/(2*r)]])

In [34]:
M_tilde.inv()*E_tilde*Matrix([[tau_r],[ tau_l]])

Matrix([
[         tau_l/(m*r) + tau_r/(m*r)],
[-L*tau_l/(2*J*r) + L*tau_r/(2*J*r)]])